## Jupyter notebook for creating links between CHi-C, ATAC-seq and RNA-seq

This notebook is the code for linking CHi-C, ATAC-seq and RNA-seq time course data used in the paper **\"Analysis of chromatin organinization and gene expression in T cells identifies functional genes for rheumatoid arthritis\"** by *Jing Yang, Amanda McGovern, Paul Martin, Kate Duffus, Xiangyu Ge, Peyman Zarrineh, Andrew P Morris, Antony Adamson, Peter Fraser, Magnus Rattray & Stephen Eyre* 

Author : *Jing Yang*  <br />
Date: 01-05-2020 <br />
For any questions about the code, please drop me a line at Jing.Yang@manchester.ac.uk

In [1]:
library(ggplot2)
library(gridExtra)
library(plyr)
library(RColorBrewer)
library(reshape2)
library(tidyverse)
library(GenomicRanges)
library(gtable)


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ tibble  3.0.1     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0
✔ purrr   0.3.4     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::arrange()   masks plyr::arrange()
✖ dplyr::combine()   masks gridExtra::combine()
✖ purrr::compact()   masks plyr::compact()
✖ dplyr::count()     masks plyr::count()
✖ dplyr::failwith()  masks plyr::failwith()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::id()        masks plyr::id()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::mutate()    masks plyr::mutate()
✖ dplyr::rename()    masks plyr::rename()
✖ dplyr::summarise() masks plyr::summarise()
✖ dplyr::summarize() masks plyr::summarize()

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘packag

### capture hic data (with replicates)

In [2]:

chic_data <- read.table("data/CHiC_data_withENSG.txt", header=TRUE)


In [3]:
dim(unique(chic_data[,c('baitID','otherEndID')]))

[1] 253100      2

In [4]:
head(chic_data,n=5)

,baitID,baitstart,baitend,ENSG,otherEndID,otherEndchr,otherEndstart,otherEndend,T01,T02,T201,T202,T1H1,T1H2,T4H1,T4H2,T24H1,baitchr,corr_rep
,<int>,<int>,<int>,<fct>,<int>,<fct>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>
1,100955,122950347,122956207,ENSG00000119965,92154,chr10,92572726,92594937,0.000000,2.281339,0.000000,0.000000,0,2.783558,2.065419,0.000000,0.000000,chr10,-0.57175609
2,100955,122950347,122956207,ENSG00000119965,78222,chr10,45508633,45514336,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,2.261001,0.000000,chr10,NA
3,100955,122950347,122956207,ENSG00000119965,77679,chr10,43644531,43648045,0.000000,2.620208,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,chr10,NA
4,100955,122950347,122956207,ENSG00000119965,101495,chr10,124901900,124920455,4.112264,3.653503,3.219815,2.623433,0,3.380836,2.881882,3.102066,0.000000,chr10,-0.07292586
5,100955,122950347,122956207,ENSG00000119965,89558,chr10,84128125,84134471,0.000000,0.000000,0.000000,2.034669,0,0.000000,1.541680,0.000000,1.930548,chr10,-0.33333333


In [5]:
names(chic_data)

[1] "baitID"        "baitstart"     "baitend"       "ENSG"         
 [5] "otherEndID"    "otherEndchr"   "otherEndstart" "otherEndend"  
 [9] "T01"           "T02"           "T201"          "T202"         
[13] "T1H1"          "T1H2"          "T4H1"          "T4H2"         
[17] "T24H1"         "baitchr"       "corr_rep"

In [6]:
dim(chic_data)

[1] 290380     19

In [7]:
names(chic_data)

[1] "baitID"        "baitstart"     "baitend"       "ENSG"         
 [5] "otherEndID"    "otherEndchr"   "otherEndstart" "otherEndend"  
 [9] "T01"           "T02"           "T201"          "T202"         
[13] "T1H1"          "T1H2"          "T4H1"          "T4H2"         
[17] "T24H1"         "baitchr"       "corr_rep"

In [8]:
dim(unique(chic_data[,c('baitID','otherEndID')]))

[1] 253100      2

In [9]:
chic_data$T0 <- 0.5*(chic_data$T01 + chic_data$T02)
chic_data$T20 <- 0.5*(chic_data$T201 + chic_data$T202)
chic_data$T1H <- 0.5*(chic_data$T1H1 + chic_data$T1H2)
chic_data$T4H <- 0.5*(chic_data$T4H1 + chic_data$T4H2)
chic_data$T24H <- chic_data$T24H1


In [10]:
cat("There are ", dim(chic_data)[1], "paried CHiC-gene datasets, among which there are ", dim(unique(chic_data[,c('baitID', 'otherEndID')]))[1], " CHIC interactions 
", length(unique(chic_data$baitID)), "baits, ", length(unique(chic_data$otherEndID)), "otherEnd and ",length(unique(chic_data$ENSG)), "ENSGs")

There are  290380 paried CHiC-gene datasets, among which there are  253100  CHIC interactions 
 6795 baits,  120621 otherEnd and  4903 ENSGs

### ATACSeq data (with replicates)

In [11]:
atacseq_data <- read.table('data/ATACSeq_data.txt', header=T)

In [12]:
head(atacseq_data)

,ATACchr,ATACstart,ATACend,ATACannotation,AT0,AT20,AT1H,AT2H,AT4H,AT24H
,<fct>,<int>,<int>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,chr10,100006331,100006563,Intron,5.004126,4.659345,5.928870,5.524712,7.401195,6.763596
2,chr10,100009362,100010421,Promoter,8.910829,8.899573,9.033632,8.848950,9.088016,8.780725
3,chr10,100118128,100118310,Intergenic,3.677326,5.423599,3.774964,5.181859,3.997616,4.538095
4,chr10,100148537,100149145,Downstream,6.308615,5.997314,6.371118,7.022634,8.320145,8.144691
5,chr10,100164173,100164282,Intron,3.793428,3.296516,4.600715,4.917097,3.838937,5.077424
6,chr10,100167603,100168094,Intron,6.121348,5.530762,5.770208,6.469869,6.229512,5.807280


In [13]:
names(atacseq_data)

[1] "ATACchr"        "ATACstart"      "ATACend"        "ATACannotation"
 [5] "AT0"            "AT20"           "AT1H"           "AT2H"          
 [9] "AT4H"           "AT24H"

In [14]:
dim(atacseq_data)

[1] 74583    10

In [15]:
atacseq_data_nopromoter <- atacseq_data[atacseq_data$Aannotation!='Promoter',]

In [16]:
cat("There are", dim(atacseq_data)[1], "peaks in Atacseq data  (minoverlap=1), among which", dim(atacseq_data_nopromoter)[1], "peaks are not within promoter region\n")

There are 74583 peaks in Atacseq data  (minoverlap=1), among which 0 peaks are not within promoter region


In [17]:
otherEnd_forcompare_data <- unique(chic_data[,c('otherEndID','otherEndchr','otherEndstart','otherEndend')])
otherEnd_forcompare_withlabel <- cbind(otherEnd_forcompare_data,rep('otherEnd',dim(otherEnd_forcompare_data)[1]))
names(otherEnd_forcompare_withlabel)<- c('ID','chr','start','end','label')
head(otherEnd_forcompare_withlabel)

,ID,chr,start,end,label
,<int>,<fct>,<int>,<int>,<fct>
1,92154,chr10,92572726,92594937,otherEnd
2,78222,chr10,45508633,45514336,otherEnd
3,77679,chr10,43644531,43648045,otherEnd
4,101495,chr10,124901900,124920455,otherEnd
5,89558,chr10,84128125,84134471,otherEnd
6,101383,chr10,124409095,124414046,otherEnd


In [18]:
dim(otherEnd_forcompare_data)

[1] 120621      4

### link ATACSeq to otherEnd data 

In [19]:
atacseq_forcompare_withlabel <- unique(cbind(1:dim(atacseq_data)[1],atacseq_data[,c(
    'ATACchr','ATACstart','ATACend')],rep('atacseq',dim(atacseq_data)[1])))
names(atacseq_forcompare_withlabel)<- c('ID','chr','start','end','label')
head(atacseq_forcompare_withlabel)

,ID,chr,start,end,label
,<int>,<fct>,<int>,<int>,<fct>
1,1,chr10,100006331,100006563,atacseq
2,2,chr10,100009362,100010421,atacseq
3,3,chr10,100118128,100118310,atacseq
4,4,chr10,100148537,100149145,atacseq
5,5,chr10,100164173,100164282,atacseq
6,6,chr10,100167603,100168094,atacseq


In [20]:
combined_atacseq_otherEnd <- rbind(atacseq_forcompare_withlabel,otherEnd_forcompare_withlabel)

In [21]:
atacseq_otherEnd_order <- combined_atacseq_otherEnd[with(combined_atacseq_otherEnd, order(combined_atacseq_otherEnd[,'chr'], combined_atacseq_otherEnd[,'start'])),]

In [22]:
len_combined <- dim(atacseq_otherEnd_order)[1]
print(len_combined)

[1] 195204


In [23]:
idx_otherEnd2atacseq <- vector(mode="list", length=dim(otherEnd_forcompare_data)[1])
idx_otherEnd <- which(atacseq_otherEnd_order$label=='otherEnd')
names(idx_otherEnd2atacseq)<-idx_otherEnd

In [24]:
idx_otherEnd_picked <- list()
idx_atacseq_picked <- list()

In [25]:
region <- 0 ## 0kb up or down the selected otherEnd data, so that ATACSeq peaks inside the fragment
range <- 200 ## index search range, [-100,100]
jj <- 1
for (ii in idx_otherEnd){
    
    min_idx <- max(1,ii-range)
    max_idx <- min(ii+range,len_combined)
    idx_range <- which(atacseq_otherEnd_order$chr[min_idx:max_idx]==atacseq_otherEnd_order$chr[ii]) + min_idx -1 
    min_start <- atacseq_otherEnd_order$start[ii]-region
    max_end <- atacseq_otherEnd_order$end[ii]+region
    #print(idx_range)
    #print(atacseq_otherEnd_order$start[idx_range])
    #print(min_start)
    start_idx <- which(atacseq_otherEnd_order$start[idx_range] >= min_start)
    end_idx <- tail(which(atacseq_otherEnd_order$end[idx_range] <= max_end),n=1)
    #print(start_idx)
    #print(end_idx)
    
    if (length(start_idx) & length(end_idx)){
        idx1 <- start_idx[1] + idx_range[1]-1
        idx2 <- tail(end_idx,n=1) + idx_range[1]-1
    idx_atacseq <- which(atacseq_otherEnd_order$label[idx1:idx2]=='atacseq') + idx1[1] -1 
    #print(ii)
    #print(idx_atacseq)
    if (length(idx_atacseq)>0){
        idx_otherEnd_picked <- c(idx_otherEnd_picked, rep(ii,length(idx_atacseq)))
        idx_atacseq_picked <- c(idx_atacseq_picked, idx_atacseq)
        idx_otherEnd2atacseq[[as.character(ii)]] <- as.list(idx_atacseq)}
    }
} 

In [26]:
idx_otherEnd2atacseq_effec <- idx_otherEnd2atacseq[-which(sapply(idx_otherEnd2atacseq,is.null))]

In [27]:
cat("There are", length(idx_otherEnd2atacseq_effec), "otherEnd interactions with atacseq peak in +/-", region)

There are 19361 otherEnd interactions with atacseq peak in +/- 0

In [28]:
tmp_otherEnd <-  atacseq_otherEnd_order[unlist(idx_otherEnd_picked, use.names=FALSE),1:4]
tmp_atacseq <- atacseq_otherEnd_order[unlist(idx_atacseq_picked, use.names=FALSE),1:4]
names(tmp_otherEnd) <- c('otherEndID','otherEndchr','otherEndstart','otherEndend')
names(tmp_atacseq) <- c('ID','ATACchr','ATACstart','ATACend')
tmp_otherEnd_atacseq_picked <- cbind(tmp_otherEnd,tmp_atacseq)

In [29]:
head(tmp_otherEnd_atacseq_picked)

,otherEndID,otherEndchr,otherEndstart,otherEndend,ID,ATACchr,ATACstart,ATACend
,<int>,<fct>,<int>,<int>,<int>,<fct>,<int>,<int>
237538,212,chr1,811328,817481,30072,chr1,817253,817413
724941,242,chr1,968260,992014,34539,chr1,975988,976499
724941.1,242,chr1,968260,992014,34614,chr1,984284,984405
724751,243,chr1,992014,1001574,34732,chr1,999291,999403
724751.1,243,chr1,992014,1001574,34739,chr1,999952,1000663
724911,246,chr1,1008296,1021819,3724,chr1,1012932,1014523


In [30]:
chic_atacseq_merged_picked <- unique(merge(merge(tmp_otherEnd_atacseq_picked,chic_data),merge(tmp_otherEnd_atacseq_picked,atacseq_data)))


In [31]:
head(chic_atacseq_merged_picked)


,otherEndID,otherEndchr,otherEndstart,otherEndend,ID,ATACchr,ATACstart,ATACend,baitID,baitstart,⋯,T1H,T4H,T24H,ATACannotation,AT0,AT20,AT1H,AT2H,AT4H,AT24H
,<int>,<fct>,<int>,<int>,<int>,<fct>,<int>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,100003,chr10,119702248,119714492,653,chr10,119705280,119705704,102382,127986294,⋯,0.0000000,1.032709,2.174726,Intergenic,5.683413,5.598023,5.945469,5.562361,6.423752,7.890997
2,100008,chr10,119724542,119727941,654,chr10,119725154,119726711,99879,119077883,⋯,1.7661949,2.691034,2.565841,Promoter,9.683430,9.417568,9.715276,9.584663,9.609715,9.586125
3,100034,chr10,119824560,119831806,661,chr10,119828019,119828186,99879,119077883,⋯,2.5941914,2.354921,2.727703,3'UTR,3.677326,4.891543,4.105506,4.974079,4.872013,4.802890
4,100034,chr10,119824560,119831806,661,chr10,119828019,119828186,77614,43386197,⋯,0.0000000,0.000000,1.636451,3'UTR,3.677326,4.891543,4.105506,4.974079,4.872013,4.802890
5,100044,chr10,119857258,119859791,662,chr10,119859344,119859770,80031,51695582,⋯,0.9665639,0.000000,0.000000,Intron,6.308615,6.139551,5.788724,6.279547,6.229512,5.776813
6,100044,chr10,119857258,119859791,662,chr10,119859344,119859770,99891,119161451,⋯,3.2718757,2.907554,3.341844,Intron,6.308615,6.139551,5.788724,6.279547,6.229512,5.776813


In [32]:
dim(chic_atacseq_merged_picked)

[1] 81971    35

In [33]:
names(chic_atacseq_merged_picked)

[1] "otherEndID"     "otherEndchr"    "otherEndstart"  "otherEndend"   
 [5] "ID"             "ATACchr"        "ATACstart"      "ATACend"       
 [9] "baitID"         "baitstart"      "baitend"        "ENSG"          
[13] "T01"            "T02"            "T201"           "T202"          
[17] "T1H1"           "T1H2"           "T4H1"           "T4H2"          
[21] "T24H1"          "baitchr"        "corr_rep"       "T0"            
[25] "T20"            "T1H"            "T4H"            "T24H"          
[29] "ATACannotation" "AT0"            "AT20"           "AT1H"          
[33] "AT2H"           "AT4H"           "AT24H"

### gene counts data (with replicates)

In [34]:
gene_data <- read.table('data/Gene_data.txt', header=T)

In [35]:
head(gene_data)

,ENSG,ENSGchr,ENSGstart,ENSGend,ET01,ET201,ET601,ET2H1,ET4H1,ET24H1,ET02,ET202,ET602,ET2H2,ET4H2,ET24H2,GeneName
,<fct>,<fct>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,ENSG00000000003,chrX,100627109,100639991,5.041981,5.062369,4.178733,3.377680,1.986341,5.324410,5.177855,3.688929,3.295454,5.253633,4.226941,4.335357,TSPAN6
2,ENSG00000000419,chr20,50934867,50958555,10.556373,11.059740,10.784106,10.332645,10.273519,10.419062,11.032946,11.032608,10.768947,10.781390,10.590316,10.778582,DPM1
3,ENSG00000000457,chr1,169849631,169894267,8.074191,7.582473,9.025557,9.324986,9.353563,9.026321,7.985902,8.162494,9.417072,9.456439,9.271116,8.889596,SCYL3
4,ENSG00000000460,chr1,169662007,169854080,9.029226,8.655751,9.993021,12.866744,14.215491,10.871978,8.287388,8.345116,10.056483,11.656388,13.905748,11.378513,C1orf112
5,ENSG00000000938,chr1,27612064,27635277,9.680082,9.113586,9.108392,9.024590,9.395413,7.303896,5.479472,5.918468,5.355478,4.750553,5.187896,2.535247,FGR
6,ENSG00000000971,chr1,196651878,196747504,6.360314,5.424491,5.394533,5.269632,4.710372,4.385639,5.551854,5.941924,5.636894,5.042941,4.976733,4.017008,CFH


In [36]:
names(gene_data)

[1] "ENSG"      "ENSGchr"   "ENSGstart" "ENSGend"   "ET01"      "ET201"    
 [7] "ET601"     "ET2H1"     "ET4H1"     "ET24H1"    "ET02"      "ET202"    
[13] "ET602"     "ET2H2"     "ET4H2"     "ET24H2"    "GeneName"

In [37]:
gene_data$ET0_a <- 0.5*(gene_data$ET01+gene_data$ET02)
gene_data$ET20_a <- 0.5*(gene_data$ET201+gene_data$ET202)
gene_data$ET1H_a <- 0.5*(gene_data$ET601+gene_data$ET602)
gene_data$ET2H_a <- 0.5*(gene_data$ET2H1+gene_data$ET2H2)
gene_data$ET4H_a <- 0.5*(gene_data$ET4H1+gene_data$ET4H2)
gene_data$ET24H_a <- 0.5*(gene_data$ET24H1+gene_data$ET24H2)

In [38]:
names(gene_data)

[1] "ENSG"      "ENSGchr"   "ENSGstart" "ENSGend"   "ET01"      "ET201"    
 [7] "ET601"     "ET2H1"     "ET4H1"     "ET24H1"    "ET02"      "ET202"    
[13] "ET602"     "ET2H2"     "ET4H2"     "ET24H2"    "GeneName"  "ET0_a"    
[19] "ET20_a"    "ET1H_a"    "ET2H_a"    "ET4H_a"    "ET24H_a"

### merge gene and chic data

In [39]:
head(chic_data)

,baitID,baitstart,baitend,ENSG,otherEndID,otherEndchr,otherEndstart,otherEndend,T01,T02,⋯,T4H1,T4H2,T24H1,baitchr,corr_rep,T0,T20,T1H,T4H,T24H
,<int>,<int>,<int>,<fct>,<int>,<fct>,<int>,<int>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,100955,122950347,122956207,ENSG00000119965,92154,chr10,92572726,92594937,0.000000,2.281339,⋯,2.065419,0.000000,0.000000,chr10,-0.57175609,1.140669,0.000000,1.391779,1.0327093,0.000000
2,100955,122950347,122956207,ENSG00000119965,78222,chr10,45508633,45514336,0.000000,0.000000,⋯,0.000000,2.261001,0.000000,chr10,NA,0.000000,0.000000,0.000000,1.1305003,0.000000
3,100955,122950347,122956207,ENSG00000119965,77679,chr10,43644531,43648045,0.000000,2.620208,⋯,0.000000,0.000000,0.000000,chr10,NA,1.310104,0.000000,0.000000,0.0000000,0.000000
4,100955,122950347,122956207,ENSG00000119965,101495,chr10,124901900,124920455,4.112264,3.653503,⋯,2.881882,3.102066,0.000000,chr10,-0.07292586,3.882884,2.921624,1.690418,2.9919739,0.000000
5,100955,122950347,122956207,ENSG00000119965,89558,chr10,84128125,84134471,0.000000,0.000000,⋯,1.541680,0.000000,1.930548,chr10,-0.33333333,0.000000,1.017335,0.000000,0.7708398,1.930548
6,100955,122950347,122956207,ENSG00000119965,101383,chr10,124409095,124414046,3.568419,3.323323,⋯,3.310275,2.741975,3.438559,chr10,0.61479564,3.445871,3.466919,2.956566,3.0261252,3.438559


In [40]:
chic_gene_mergedbyENSG <- merge(gene_data,chic_data)

In [42]:
dim(chic_gene_mergedbyENSG)

[1] 259208     46

In [43]:
names(chic_gene_mergedbyENSG)

[1] "ENSG"          "ENSGchr"       "ENSGstart"     "ENSGend"      
 [5] "ET01"          "ET201"         "ET601"         "ET2H1"        
 [9] "ET4H1"         "ET24H1"        "ET02"          "ET202"        
[13] "ET602"         "ET2H2"         "ET4H2"         "ET24H2"       
[17] "GeneName"      "ET0_a"         "ET20_a"        "ET1H_a"       
[21] "ET2H_a"        "ET4H_a"        "ET24H_a"       "baitID"       
[25] "baitstart"     "baitend"       "otherEndID"    "otherEndchr"  
[29] "otherEndstart" "otherEndend"   "T01"           "T02"          
[33] "T201"          "T202"          "T1H1"          "T1H2"         
[37] "T4H1"          "T4H2"          "T24H1"         "baitchr"      
[41] "corr_rep"      "T0"            "T20"           "T1H"          
[45] "T4H"           "T24H"

### link ATACSeq to gene replicate

In [44]:
head(tmp_otherEnd_atacseq_picked)
head(chic_gene_mergedbyENSG)

,otherEndID,otherEndchr,otherEndstart,otherEndend,ID,ATACchr,ATACstart,ATACend
,<int>,<fct>,<int>,<int>,<int>,<fct>,<int>,<int>
237538,212,chr1,811328,817481,30072,chr1,817253,817413
724941,242,chr1,968260,992014,34539,chr1,975988,976499
724941.1,242,chr1,968260,992014,34614,chr1,984284,984405
724751,243,chr1,992014,1001574,34732,chr1,999291,999403
724751.1,243,chr1,992014,1001574,34739,chr1,999952,1000663
724911,246,chr1,1008296,1021819,3724,chr1,1012932,1014523


,ENSG,ENSGchr,ENSGstart,ENSGend,ET01,ET201,ET601,ET2H1,ET4H1,ET24H1,⋯,T4H1,T4H2,T24H1,baitchr,corr_rep,T0,T20,T1H,T4H,T24H
,<fct>,<fct>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,ENSG00000001084,chr6,53497341,53616970,9.089916,9.237243,9.0537,8.633409,9.227341,9.696194,⋯,0.000000,0.000000,1.266637,chr6,NA,0.000000,0.000000,0.000000,0.000000,1.266637
2,ENSG00000001084,chr6,53497341,53616970,9.089916,9.237243,9.0537,8.633409,9.227341,9.696194,⋯,4.375070,4.175490,4.863939,chr6,0.7445711,3.858361,3.722884,4.446843,4.275280,4.863939
3,ENSG00000001084,chr6,53497341,53616970,9.089916,9.237243,9.0537,8.633409,9.227341,9.696194,⋯,5.119641,4.970205,4.964815,chr6,-0.9443833,5.079635,4.912067,5.132516,5.044923,4.964815
4,ENSG00000001084,chr6,53497341,53616970,9.089916,9.237243,9.0537,8.633409,9.227341,9.696194,⋯,4.951598,4.832939,5.408610,chr6,0.7553095,4.832475,5.588350,5.245855,4.892268,5.408610
5,ENSG00000001084,chr6,53497341,53616970,9.089916,9.237243,9.0537,8.633409,9.227341,9.696194,⋯,2.751463,2.521440,3.005400,chr6,0.9927271,3.043141,3.013895,3.521647,2.636451,3.005400
6,ENSG00000001084,chr6,53497341,53616970,9.089916,9.237243,9.0537,8.633409,9.227341,9.696194,⋯,0.000000,0.000000,0.000000,chr6,NA,0.000000,0.000000,1.088724,0.000000,0.000000


In [45]:
head(chic_gene_mergedbyENSG)

,ENSG,ENSGchr,ENSGstart,ENSGend,ET01,ET201,ET601,ET2H1,ET4H1,ET24H1,⋯,T4H1,T4H2,T24H1,baitchr,corr_rep,T0,T20,T1H,T4H,T24H
,<fct>,<fct>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,ENSG00000001084,chr6,53497341,53616970,9.089916,9.237243,9.0537,8.633409,9.227341,9.696194,⋯,0.000000,0.000000,1.266637,chr6,NA,0.000000,0.000000,0.000000,0.000000,1.266637
2,ENSG00000001084,chr6,53497341,53616970,9.089916,9.237243,9.0537,8.633409,9.227341,9.696194,⋯,4.375070,4.175490,4.863939,chr6,0.7445711,3.858361,3.722884,4.446843,4.275280,4.863939
3,ENSG00000001084,chr6,53497341,53616970,9.089916,9.237243,9.0537,8.633409,9.227341,9.696194,⋯,5.119641,4.970205,4.964815,chr6,-0.9443833,5.079635,4.912067,5.132516,5.044923,4.964815
4,ENSG00000001084,chr6,53497341,53616970,9.089916,9.237243,9.0537,8.633409,9.227341,9.696194,⋯,4.951598,4.832939,5.408610,chr6,0.7553095,4.832475,5.588350,5.245855,4.892268,5.408610
5,ENSG00000001084,chr6,53497341,53616970,9.089916,9.237243,9.0537,8.633409,9.227341,9.696194,⋯,2.751463,2.521440,3.005400,chr6,0.9927271,3.043141,3.013895,3.521647,2.636451,3.005400
6,ENSG00000001084,chr6,53497341,53616970,9.089916,9.237243,9.0537,8.633409,9.227341,9.696194,⋯,0.000000,0.000000,0.000000,chr6,NA,0.000000,0.000000,1.088724,0.000000,0.000000


In [46]:
head(tmp_otherEnd_atacseq_picked)

,otherEndID,otherEndchr,otherEndstart,otherEndend,ID,ATACchr,ATACstart,ATACend
,<int>,<fct>,<int>,<int>,<int>,<fct>,<int>,<int>
237538,212,chr1,811328,817481,30072,chr1,817253,817413
724941,242,chr1,968260,992014,34539,chr1,975988,976499
724941.1,242,chr1,968260,992014,34614,chr1,984284,984405
724751,243,chr1,992014,1001574,34732,chr1,999291,999403
724751.1,243,chr1,992014,1001574,34739,chr1,999952,1000663
724911,246,chr1,1008296,1021819,3724,chr1,1012932,1014523


In [47]:

atacseq_gene <- unique(merge(merge(tmp_otherEnd_atacseq_picked,chic_gene_mergedbyENSG),atacseq_data))

In [48]:
dim(atacseq_gene)

[1] 76914    57

In [49]:
head(atacseq_gene)

,ATACchr,ATACstart,ATACend,otherEndID,otherEndchr,otherEndstart,otherEndend,ID,ENSG,ENSGchr,⋯,T1H,T4H,T24H,ATACannotation,AT0,AT20,AT1H,AT2H,AT4H,AT24H
,<fct>,<int>,<int>,<int>,<fct>,<int>,<int>,<int>,<fct>,<fct>,⋯,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,chr10,100168185,100168481,94270,chr10,100168142,100171879,7,ENSG00000122873,chr10,⋯,0.000000,2.004188,0.000000,Intron,5.185079,5.386033,5.549115,5.081706,5.133666,5.469091
2,chr10,100168185,100168481,94270,chr10,100168142,100171879,7,ENSG00000099204,chr10,⋯,0.000000,0.000000,1.930548,Intron,5.185079,5.386033,5.549115,5.081706,5.133666,5.469091
3,chr10,100267287,100268052,94301,chr10,100265170,100287223,11,ENSG00000052749,chr10,⋯,0.000000,0.000000,2.174726,Promoter,7.772423,7.507165,7.616713,7.874698,7.849217,7.668432
4,chr10,100286005,100287018,94301,chr10,100265170,100287223,12,ENSG00000052749,chr10,⋯,0.000000,0.000000,2.174726,Promoter,9.086124,8.986858,9.306845,9.077411,9.375033,8.877224
5,chr10,100481544,100482916,94352,chr10,100476882,100482954,21,ENSG00000213341,chr10,⋯,2.480503,1.799894,1.636451,3'UTR,9.147414,9.275847,9.463431,8.709092,9.302763,8.877224
6,chr10,100508684,100509527,94364,chr10,100507722,100518706,22,ENSG00000052749,chr10,⋯,0.000000,0.000000,2.174726,Promoter,8.054279,8.223312,8.532925,7.674360,8.887220,8.168356


In [50]:
cat("There are ", dim(atacseq_gene)[1], "paried bait-otherEnd-atacseq-gene datasets, among which ", 
    dim(unique(atacseq_gene[,c('baitID', 'ID')]))[1], "paried bait-ATACSeq links; ", 
    dim(unique(atacseq_gene[,c('ID', 'otherEndID')]))[1], "ATACSeq-otherEnd links; ", 
    dim(unique(atacseq_gene[,c('ID', 'ENSG')]))[1], "ATACSeq-gene links; ", 
   dim(unique(atacseq_gene[,c('baitID', 'ENSG')]))[1], " bait-gene links.  There are ", dim(unique(atacseq_gene[,c('baitID', 'otherEndID')]))[1], " CHIC interactions, 
", length(unique(atacseq_gene$baitID)), "baits, ", length(unique(atacseq_gene$otherEndID)), "otherEnd, ",length(unique(atacseq_gene$ENSG)), "ENSGs and ", length(unique(atacseq_gene$ID)), "ATACSeq peaks")

There are  76914 paried bait-otherEnd-atacseq-gene datasets, among which  68369 paried bait-ATACSeq links;  24295 ATACSeq-otherEnd links;  65827 ATACSeq-gene links;  5706  bait-gene links.  There are  52883  CHIC interactions, 
 5148 baits,  18898 otherEnd,  3516 ENSGs and  24295 ATACSeq peaks

In [51]:


atacseq_gene$corr_atacseq_gene <- sapply(seq.int(dim(atacseq_gene)[1]),function(i) cor(t(atacseq_gene[i,c('AT0','AT20','AT1H','AT2H','AT4H','AT24H')]),t(atacseq_gene[i,c('ET0_a','ET20_a','ET1H_a','ET2H_a','ET4H_a','ET24H_a')])))
atacseq_gene$corr_atacseq_otherEnd <- sapply(seq.int(dim(atacseq_gene)[1]),function(i) cor(t(atacseq_gene[i,c('AT0','AT20','AT1H','AT4H','AT24H')]),t(atacseq_gene[i,c('T0','T20','T1H','T4H','T24H')])))
atacseq_gene$corr_gene_otherEnd <- sapply(seq.int(dim(atacseq_gene)[1]),function(i) cor(t(atacseq_gene[i,c('ET0_a','ET20_a','ET1H_a','ET4H_a','ET24H_a')]),t(atacseq_gene[i,c('T0','T20','T1H','T4H','T24H')])))


In [52]:
atacseq_gene_nopromoter <- atacseq_gene[atacseq_gene$ATACannotation!='Promoter',]
dim(atacseq_gene)
dim(atacseq_gene_nopromoter)

[1] 76914    60

[1] 44113    60

In [53]:
atacseq_BICandLR <- read.table('data/atacseq_BICandLR.txt', header=T)
chic_BICandLR <- read.table('data/chic_BICandLR.txt', header=T)
gene_BICandLR <- read.table('data/gene_BICandLR.txt', header=T)

In [54]:
head(atacseq_BICandLR)

,ATACchr,ATACstart,ATACend,AT0,AT20,AT1H,AT2H,AT4H,AT24H,BIC_atacseq,LR_atacseq
,<fct>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,chr10,100168185,100168481,5.185079,5.386033,5.549115,5.081706,5.133666,5.469091,1.791775,1.592097e-05
2,chr10,100267287,100268052,7.772423,7.507165,7.616713,7.874698,7.849217,7.668432,0.991704,-8.000554e-01
3,chr10,100286005,100287018,9.086124,8.986858,9.306845,9.077411,9.375033,8.877224,1.791826,6.613878e-05
4,chr10,100481544,100482916,9.147414,9.275847,9.463431,8.709092,9.302763,8.877224,1.791772,1.285027e-05
5,chr10,100508684,100509527,8.054279,8.223312,8.532925,7.674360,8.887220,8.168356,1.791769,9.747732e-06
6,chr10,100510838,100511373,8.952013,9.005558,9.971759,8.586541,9.457576,8.248255,1.791792,3.244872e-05


In [56]:
atacseq_gene_withBICandLR <- left_join(left_join(left_join(atacseq_gene, atacseq_BICandLR), chic_BICandLR), gene_BICandLR)

Joining, by = c("ATACchr", "ATACstart", "ATACend", "AT0", "AT20", "AT1H", "AT2H", "AT4H", "AT24H")

Joining, by = c("otherEndID", "baitID", "T01", "T02", "T201", "T202", "T1H1", "T1H2", "T4H1", "T4H2", "T24H1")

Joining, by = c("ENSG", "ET01", "ET201", "ET601", "ET2H1", "ET4H1", "ET24H1", "ET02", "ET202", "ET602", "ET2H2", "ET4H2", "ET24H2")



In [57]:
names(atacseq_gene_withBICandLR)

[1] "ATACchr"               "ATACstart"             "ATACend"              
 [4] "otherEndID"            "otherEndchr"           "otherEndstart"        
 [7] "otherEndend"           "ID"                    "ENSG"                 
[10] "ENSGchr"               "ENSGstart"             "ENSGend"              
[13] "ET01"                  "ET201"                 "ET601"                
[16] "ET2H1"                 "ET4H1"                 "ET24H1"               
[19] "ET02"                  "ET202"                 "ET602"                
[22] "ET2H2"                 "ET4H2"                 "ET24H2"               
[25] "GeneName"              "ET0_a"                 "ET20_a"               
[28] "ET1H_a"                "ET2H_a"                "ET4H_a"               
[31] "ET24H_a"               "baitID"                "baitstart"            
[34] "baitend"               "T01"                   "T02"                  
[37] "T201"                  "T202"                  "T1H1"                 
[40] "T1H2"                  "T4H1"                  "T4H2"                 
[43] "T24H1"                 "baitchr"               "corr_rep"             
[46] "T0"                    "T20"                   "T1H"                  
[49] "T4H"                   "T24H"                  "ATACannotation"       
[52] "AT0"                   "AT20"                  "AT1H"                 
[55] "AT2H"                  "AT4H"                  "AT24H"                
[58] "corr_atacseq_gene"     "corr_atacseq_otherEnd" "corr_gene_otherEnd"   
[61] "BIC_atacseq"           "LR_atacseq"            "BIC_chic"             
[64] "LR_chic"               "BIC_gene"              "LR_gene"

In [59]:
write.table(atacseq_gene_withBICandLR, file='data/ATACseq_CHiC_RNAseq_linked_withpromoter.txt', quote=F, row.names=F)